In [7]:
import speech_recognition as sr
from transformers import pipeline

# 음성 인식 함수
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("말씀해주세요...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language="ko-KR")  # 한국어 인식
        print(f"인식된 텍스트: {text}")
        return text
    except sr.UnknownValueError:
        print("음성을 이해할 수 없습니다.")
        return None
    except sr.RequestError as e:
        print(f"음성 인식 서비스 오류: {e}")
        return None

# 의도 분류 함수
def classify_intent(text):
    # 사전 학습된 모델 로드
    classifier = pipeline("text-classification", model="joeddav/xlm-roberta-large-xnli")  # 다국어 지원 모델
    
    # 입력 문장 분류
    result = classifier(text)
    return result[0]  # 가장 높은 확률의 라벨 반환

# 프로그램 실행
if __name__ == "__main__":
    print("프로그램 시작")
    user_input = recognize_speech()  # 음성 -> 텍스트
    if user_input:
        intent_result = classify_intent(user_input)  # 텍스트 -> 의도 분석
        print(f"분석 결과: {intent_result}")
        if "stop" in intent_result['label'].lower():  # 예시: 멈추는 의도 분류
            print("의도: 내리고 싶다는 요청으로 판단됩니다.")
        else:
            print("의도: 내리고 싶다는 요청이 아닌 것으로 판단됩니다.")


프로그램 시작
말씀해주세요...


ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664

인식된 텍스트: 나 배고파


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


분석 결과: {'label': 'entailment', 'score': 0.7272838354110718}
의도: 내리고 싶다는 요청이 아닌 것으로 판단됩니다.


In [1]:
import sounddevice as sd
import numpy as np
import whisper
import tempfile
import os
from transformers import pipeline

# 1. 마이크로 음성 녹음
def record_audio_from_mic(duration=5, fs=16000):
    """
    마이크로 음성을 녹음하여 임시 파일로 저장합니다.
    Args:
        duration (int): 녹음 시간 (초).
        fs (int): 샘플링 속도 (기본값 16000Hz).
    Returns:
        str: 저장된 임시 WAV 파일 경로.
    """
    print("녹음 시작... 말해주세요!")
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # 녹음 종료 대기
    print("녹음 완료!")

    # 임시 파일 생성
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    with open(temp_file.name, 'wb') as f:
        import wave
        wf = wave.open(f, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(audio_data.tobytes())
        wf.close()

    return temp_file.name

# 2. Whisper로 음성을 텍스트로 변환
def transcribe_audio_with_whisper(audio_path):
    """
    Whisper를 사용하여 오디오 파일을 텍스트로 변환합니다.
    Args:
        audio_path (str): 오디오 파일 경로.
    Returns:
        str: 변환된 텍스트.
    """
    model = whisper.load_model("base")
    result = model.transcribe(audio_path, language="ko")
    return result["text"]

# 3. Hugging Face 모델을 사용한 의도 분석
def analyze_intent_with_accuracy(text):
    """
    Hugging Face 모델을 사용하여 텍스트의 의도를 분석하고 확률을 표시합니다.
    Args:
        text (str): 변환된 텍스트.
    Returns:
        tuple: ("STOP_REQUEST" 또는 "CONTINUE_REQUEST", 정확도 점수).
    """
    # 모델 로드
    classifier = pipeline("text-classification", model="joeddav/xlm-roberta-large-xnli")
    result = classifier(text)

    # 결과 출력
    label = result[0]["label"]
    score = result[0]["score"]

    # 확률과 함께 판단
    if label == "entailment":
        return "STOP_REQUEST", score
    return "CONTINUE_REQUEST", score

# 4. 통합 메인 함수
def main():
    duration = 5  # 녹음 시간 (초 단위)
    audio_path = record_audio_from_mic(duration)

    try:
        print("Whisper로 음성을 텍스트로 변환 중...")
        text = transcribe_audio_with_whisper(audio_path)
        print(f"변환된 텍스트: {text}")

        print("의도 분석 중...")
        intent, accuracy = analyze_intent_with_accuracy(text)
        print(f"분석된 의도: {intent}")
        print(f"정확도: {accuracy:.2f}")  # 소수점 2자리로 정확도 출력

        if intent == "STOP_REQUEST":
            print("결론: 내리고 싶다는 요청으로 판단됩니다.")
        else:
            print("결론: 내리고 싶지 않다는 요청으로 판단됩니다.")
    finally:
        # 임시 파일 삭제
        if os.path.exists(audio_path):
            os.remove(audio_path)

# 실행
if __name__ == "__main__":
    main()


2024-11-26 16:28:08.067286: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 16:28:08.106269: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 16:28:08.106321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 16:28:08.107813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 16:28:08.116193: I tensorflow/core/platform/cpu_feature_guar

녹음 시작... 말해주세요!
녹음 완료!
Whisper로 음성을 텍스트로 변환 중...


/home/kjj37/venv/machine_learning/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locatio

변환된 텍스트:  네away
의도 분석 중...


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


분석된 의도: STOP_REQUEST
정확도: 0.44
결론: 내리고 싶다는 요청으로 판단됩니다.


In [6]:
import speech_recognition as sr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# 1. 학습 데이터
texts = [
    "여기서 내려 주세요.", "이쯤에서 세워 주세요.", "지금 멈춰 주세요.", "여기 괜찮아요.", "여기서 내릴게요.",
    "잠깐만요, 여기서 내려 주세요.", "아, 여기 세워 주세요.", "저기 앞에서 멈춰 주세요.", "여기 내리겠습니다.", "여기 내려도 돼요.",
    "좀 더 가주세요.", "다음 교차로에서 멈춰 주세요.", "목적지까지 가주세요.", "아직 내릴 곳 아니에요.", "조금 더 가주세요.",
    "직진 부탁드립니다.", "좌회전해주세요.", "우회전 부탁드려요.", "목적지까지 부탁드려요.", "조금 더 빨리 가주세요."
]
labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# 데이터 벡터화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# 모델 학습
model = LogisticRegression()
model.fit(X, labels)

# 2. 음성 인식 함수
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("말씀하세요...")
        try:
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio, language="ko-KR")
            print(f"인식된 텍스트: {text}")
            return text
        except sr.UnknownValueError:
            print("음성을 이해할 수 없습니다.")
            return None
        except sr.RequestError as e:
            print(f"Google API 에러: {e}")
            return None

# 3. 의도 분류 함수
def classify_intent(text):
    if text:
        vectorized_text = vectorizer.transform([text])
        prediction = model.predict(vectorized_text)
        if prediction[0] == 1:
            print("분석 결과: 손님은 여기서 내리기를 원합니다.")
        else:
            print("분석 결과: 손님은 아직 내리기를 원하지 않습니다.")
    else:
        print("텍스트를 인식하지 못했습니다.")

# 4. 실행 코드
if __name__ == "__main__":
    print("택시 의도 분석 프로그램 시작!")
    while True:
        print("\n음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).")
        text = speech_to_text()
        if text and text.lower() == "exit":
            print("프로그램을 종료합니다.")
            break
        classify_intent(text)

택시 의도 분석 프로그램 시작!

음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).
말씀하세요...
인식된 텍스트: 아 배고프다
분석 결과: 손님은 아직 내리기를 원하지 않습니다.

음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).
말씀하세요...
음성을 이해할 수 없습니다.
텍스트를 인식하지 못했습니다.

음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).
말씀하세요...
인식된 텍스트: 아 심심해
분석 결과: 손님은 아직 내리기를 원하지 않습니다.

음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).
말씀하세요...
음성을 이해할 수 없습니다.
텍스트를 인식하지 못했습니다.

음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).
말씀하세요...
인식된 텍스트: 내리고 싶어
분석 결과: 손님은 아직 내리기를 원하지 않습니다.

음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).
말씀하세요...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import speech_recognition as sr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# 1. CSV 파일에서 데이터 불러오기
data_file = "/mnt/data/taxi_intent_dataset_with_dialogue.csv"
data = pd.read_csv(data_file)

texts = data["text"].tolist()
labels = data["label"].tolist()

# 데이터 벡터화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# 모델 학습
model = LogisticRegression()
model.fit(X, labels)

# 2. 음성 인식 함수
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("말씀하세요...")
        try:
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio, language="ko-KR")
            print(f"인식된 텍스트: {text}")
            return text
        except sr.UnknownValueError:
            print("음성을 이해할 수 없습니다.")
            return None
        except sr.RequestError as e:
            print(f"Google API 에러: {e}")
            return None

# 3. 의도 분류 함수
def classify_intent(text):
    if text:
        vectorized_text = vectorizer.transform([text])
        prediction = model.predict(vectorized_text)
        if prediction[0] == 1:
            print("분석 결과: 손님은 여기서 내리기를 원합니다.")
        elif prediction[0] == 0:
            print("분석 결과: 손님은 아직 내리기를 원하지 않습니다.")
        elif prediction[0] == 2:
            print("분석 결과: 일상 대화로 인식되었습니다.")
    else:
        print("텍스트를 인식하지 못했습니다.")

# 4. 실행 코드
if __name__ == "__main__":
    print("택시 의도 분석 프로그램 시작!")
    while True:
        print("\n음성을 입력하려면 말하세요 (종료하려면 'exit' 입력).")
        text = speech_to_text()
        if text and text.lower() == "exit":
            print("프로그램을 종료합니다.")
            break
        classify_intent(text)
